In [1]:
import os, json, re, stat, pathlib, base64
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from pathlib import Path

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
def jaccard_similarity(list1, list2):
    set1 = set(list1)
    set2 = set(list2)
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return float(intersection/union) if union !=0 else 0

In [4]:
DATA_PATH = "/Users/ethancoskay/projects/MovieRecs/data/TMDB_all_movies.csv"
df_main = pd.read_csv(DATA_PATH)
df_main.head()

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,original_language,original_title,overview,popularity,tagline,genres,production_companies,production_countries,spoken_languages,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes,poster_path
0,2,Ariel,7.100,346.0,Released,1988-10-21,0.0,73.0,0.0,tt0094675,fi,Ariel,A Finnish man goes to the city to find a job a...,1.7519,NaN,"Comedy, Drama, Romance, Crime",Villealfa Filmproductions,Finland,suomi,"Merja Pulkkinen, Eetu Hilkamo, Turo Pajala, Es...",Aki Kaurismäki,Timo Salminen,Aki Kaurismäki,Aki Kaurismäki,NaN,7.4,9148.0,/ojDg0PGvs6R9xYFodRct2kdI6wC.jpg
1,3,Shadows in Paradise,7.293,409.0,Released,1986-10-17,0.0,74.0,0.0,tt0092149,fi,Varjoja paratiisissa,"Nikander, a rubbish collector and would-be ent...",1.9295,NaN,"Comedy, Drama, Romance",Villealfa Filmproductions,Finland,"suomi, English, svenska","Esko Nikkari, Mari Rantasila, Marina Martinoff...",Aki Kaurismäki,Timo Salminen,Aki Kaurismäki,Mika Kaurismäki,NaN,7.4,7969.0,/nj01hspawPof0mJmlgfjuLyJuRN.jpg
2,5,Four Rooms,5.861,2687.0,Released,1995-12-09,4257354.0,98.0,4000000.0,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,9.6284,Twelve outrageous guests. Four scandalous requ...,"Comedy, Crime","Miramax, A Band Apart",United States of America,English,"Quinn Hellerman, Lawrence Bender, Tim Roth, La...","Allison Anders, Robert Rodriguez, Alexandre Ro...","Phil Parmet, Rodrigo García, Guillermo Navarro...","Allison Anders, Robert Rodriguez, Alexandre Ro...","Lawrence Bender, Alexandre Rockwell, Quentin T...",Combustible Edison,6.7,114082.0,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg
3,6,Judgment Night,6.500,349.0,Released,1993-10-15,12136938.0,109.0,21000000.0,tt0107286,en,Judgment Night,"Four young friends, while taking a shortcut en...",4.5607,Don't move. Don't whisper. Don't even breathe.,"Action, Crime, Thriller","Largo Entertainment, JVC, Universal Pictures",United States of America,English,"Michael Scranton, Hank McGill, Donovan D. Ross...",Stephen Hopkins,Peter Levy,"Jere Cunningham, Lewis Colick","Gene Levy, Lloyd Segan, Marilyn Vance",Alan Silvestri,6.6,19885.0,/3rvvpS9YPM5HB2f4HYiNiJVtdam.jpg
4,8,Life in Loops (A Megacities RMX),7.500,27.0,Released,2006-01-01,0.0,80.0,42000.0,tt0825671,en,Life in Loops (A Megacities RMX),Timo Novotny labels his new project an experim...,3.2030,A Megacities remix.,Documentary,inLoops,Austria,"English, हिन्दी, 日本語, Pусский, Español",NaN,Timo Novotny,Wolfgang Thaler,"Michael Glawogger, Timo Novotny","Ulrich Gehmacher, Timo Novotny",NaN,8.2,284.0,/7ln81BRnPR2wqxuITZxEciCe1lc.jpg


In [5]:
df = df_main[['id', 'title', 'overview', 'genres', 'production_companies', 'cast', 'director', 'writers', 'producers']]

In [6]:
cols_to_lower = ['director', 'writers', 'producers', 'cast', 'production_companies', 'genres']
for col in cols_to_lower:
    df[col] = df[col].fillna('').str.lower().str.replace(r"\s+", "", regex=True).str.split(",")
    df[col] = df[col].apply(lambda x: x[:4] if len(x)>4 else x)


df["overview"] = df["overview"].fillna('').str.lower()

/var/folders/39/8v7z_shs5qd71d5q4_6m776w0000gn/T/ipykernel_79288/2146218560.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna('').str.lower().str.replace(r"\s+", "", regex=True).str.split(",")
/var/folders/39/8v7z_shs5qd71d5q4_6m776w0000gn/T/ipykernel_79288/2146218560.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(lambda x: x[:4] if len(x)>4 else x)
/var/folders/39/8v7z_shs5qd71d5q4_6m776w0000gn/T/ipykernel_79288/2146218560.py:7: SettingWithCopyWarning

In [7]:
df.head()

,id,title,overview,genres,production_companies,cast,director,writers,producers
0,2,Ariel,a finnish man goes to the city to find a job a...,"[comedy, drama, romance, crime]",[villealfafilmproductions],"[merjapulkkinen, eetuhilkamo, turopajala, esko...",[akikaurismäki],[akikaurismäki],[akikaurismäki]
1,3,Shadows in Paradise,"nikander, a rubbish collector and would-be ent...","[comedy, drama, romance]",[villealfafilmproductions],"[eskonikkari, marirantasila, marinamartinoff, ...",[akikaurismäki],[akikaurismäki],[mikakaurismäki]
2,5,Four Rooms,it's ted the bellhop's first night on the job....,"[comedy, crime]","[miramax, abandapart]","[quinnhellerman, lawrencebender, timroth, lana...","[allisonanders, robertrodriguez, alexandrerock...","[allisonanders, robertrodriguez, alexandrerock...","[lawrencebender, alexandrerockwell, quentintar..."
3,6,Judgment Night,"four young friends, while taking a shortcut en...","[action, crime, thriller]","[largoentertainment, jvc, universalpictures]","[michaelscranton, hankmcgill, donovand.ross, n...",[stephenhopkins],"[jerecunningham, lewiscolick]","[genelevy, lloydsegan, marilynvance]"
4,8,Life in Loops (A Megacities RMX),timo novotny labels his new project an experim...,[documentary],[inloops],[],[timonovotny],"[michaelglawogger, timonovotny]","[ulrichgehmacher, timonovotny]"


In [8]:
df.cast[0]

['merjapulkkinen', 'eetuhilkamo', 'turopajala', 'eskosalminen']

In [9]:
cols_to_merge = ['director', 'writers', 'producers', 'cast', 'production_companies', 'genres']

# Function to join lists from multiple columns, convert to set (for uniqueness), then back to list
def merge_and_unique(row):
    combined = []
    for col in cols_to_merge:
        combined.extend(row[col])  # Add the list from each column to combined list
    return list(set(combined))  # Convert to set for uniqueness, then back to list

# Apply the function to each row
df['tags'] = df.apply(merge_and_unique, axis=1)

/var/folders/39/8v7z_shs5qd71d5q4_6m776w0000gn/T/ipykernel_79288/2250808477.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df.apply(merge_and_unique, axis=1)


In [10]:
df.drop(columns=cols_to_merge, inplace=True)

/var/folders/39/8v7z_shs5qd71d5q4_6m776w0000gn/T/ipykernel_79288/708697862.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=cols_to_merge, inplace=True)


In [11]:
df.set_index('id', inplace=True)

In [12]:
df.head()

,title,overview,tags
id,,,
2,Ariel,a finnish man goes to the city to find a job a...,"[drama, comedy, akikaurismäki, turopajala, cri..."
3,Shadows in Paradise,"nikander, a rubbish collector and would-be ent...","[comedy, akikaurismäki, eskonikkari, mikakauri..."
5,Four Rooms,it's ted the bellhop's first night on the job....,"[quentintarantino, abandapart, allisonanders, ..."
6,Judgment Night,"four young friends, while taking a shortcut en...","[donovand.ross, michaelscranton, action, lloyd..."
8,Life in Loops (A Megacities RMX),timo novotny labels his new project an experim...,"[, documentary, timonovotny, michaelglawogger,..."


In [13]:
df['tags'] = df['tags'].apply(lambda x: [item for item in x if item != ''] if isinstance(x, list) else x)
df.head()

/var/folders/39/8v7z_shs5qd71d5q4_6m776w0000gn/T/ipykernel_79288/1518854655.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(lambda x: [item for item in x if item != ''] if isinstance(x, list) else x)


,title,overview,tags
id,,,
2,Ariel,a finnish man goes to the city to find a job a...,"[drama, comedy, akikaurismäki, turopajala, cri..."
3,Shadows in Paradise,"nikander, a rubbish collector and would-be ent...","[comedy, akikaurismäki, eskonikkari, mikakauri..."
5,Four Rooms,it's ted the bellhop's first night on the job....,"[quentintarantino, abandapart, allisonanders, ..."
6,Judgment Night,"four young friends, while taking a shortcut en...","[donovand.ross, michaelscranton, action, lloyd..."
8,Life in Loops (A Megacities RMX),timo novotny labels his new project an experim...,"[documentary, timonovotny, michaelglawogger, u..."


In [14]:
df["tags"][8]

['documentary',
 'timonovotny',
 'michaelglawogger',
 'ulrichgehmacher',
 'inloops']

In [15]:
# Count the number of duplicates
df['title'].duplicated().sum()

166942

In [16]:
df = df.drop_duplicates(subset=['title'], keep='first')

In [17]:
df[df['title'].duplicated()]

,title,overview,tags
id,,,


In [18]:
df.shape

(921964, 3)

In [19]:
df.head()

,title,overview,tags
id,,,
2,Ariel,a finnish man goes to the city to find a job a...,"[drama, comedy, akikaurismäki, turopajala, cri..."
3,Shadows in Paradise,"nikander, a rubbish collector and would-be ent...","[comedy, akikaurismäki, eskonikkari, mikakauri..."
5,Four Rooms,it's ted the bellhop's first night on the job....,"[quentintarantino, abandapart, allisonanders, ..."
6,Judgment Night,"four young friends, while taking a shortcut en...","[donovand.ross, michaelscranton, action, lloyd..."
8,Life in Loops (A Megacities RMX),timo novotny labels his new project an experim...,"[documentary, timonovotny, michaelglawogger, u..."


In [20]:
df.isnull().sum()

title       1
overview    0
tags        0
dtype: int64

In [21]:
df.dropna(inplace=True)

In [22]:
df.head()

,title,overview,tags
id,,,
2,Ariel,a finnish man goes to the city to find a job a...,"[drama, comedy, akikaurismäki, turopajala, cri..."
3,Shadows in Paradise,"nikander, a rubbish collector and would-be ent...","[comedy, akikaurismäki, eskonikkari, mikakauri..."
5,Four Rooms,it's ted the bellhop's first night on the job....,"[quentintarantino, abandapart, allisonanders, ..."
6,Judgment Night,"four young friends, while taking a shortcut en...","[donovand.ross, michaelscranton, action, lloyd..."
8,Life in Loops (A Megacities RMX),timo novotny labels his new project an experim...,"[documentary, timonovotny, michaelglawogger, u..."


In [23]:
df=df[:30000]

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk

# Ensure necessary NLTK data is downloaded
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

# Initialize lemmatizer and stop words
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Define custom tokenizer
def lemmatize_and_remove_stopwords(text):
    tokens = word_tokenize(text.lower())
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    filtered_tokens = [token for token in lemmatized_tokens if token not in stop_words]
    return filtered_tokens

# Initialize TfidfVectorizer with custom tokenizer
vectorizer = TfidfVectorizer(tokenizer=lemmatize_and_remove_stopwords, max_features=5000)

tfidf_matrix = vectorizer.fit_transform(df["overview"])

# To view the feature names
print(len(vectorizer.get_feature_names_out()))


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ethancoskay/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ethancoskay/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ethancoskay/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/Applications/anaconda3/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


5000


In [25]:
cosineSimilarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [26]:
print(cosineSimilarities.shape)

(30000, 30000)


In [27]:
print(vectorizer.get_feature_names_out())

['!' '#' '$' ... '“' '”' '•']


In [28]:
def combined_similarity(idx, cos_sim_matrix, df):
    # Get the actual position of the given index
    idx_pos = df.index.get_loc(idx)

    results = []
    for i in df.index:
        i_pos = df.index.get_loc(i)

        # Calculate similarities
        jac = jaccard_similarity(df['tags'][idx], df['tags'][i])
        cos = cos_sim_matrix[idx_pos][i_pos]

        combined = (cos * 0.5 + jac * 0.5)
        results.append((i, combined))  # i is the actual index label
    
    return sorted(results, key=lambda x: x[1], reverse=True)[:11]


In [29]:
recommendations = combined_similarity(idx=122, cos_sim_matrix=cosineSimilarities, df=df)

In [30]:
recommendations

[(122, 1.0000000000000002),
 (121, 0.45933886506834143),
 (120, 0.34786420019386455),
 (1361, 0.17857830684341777),
 (123, 0.177862407684454),
 (17832, 0.16533194457981748),
 (14763, 0.14137827344094783),
 (3040, 0.1360447804088222),
 (13194, 0.13187240739177963),
 (28058, 0.13171551020624744),
 (45678, 0.12972761206468014)]

In [31]:
for pair in recommendations:
    print(df.loc[pair[0]].title)

The Lord of the Rings: The Return of the King
The Lord of the Rings: The Two Towers
The Lord of the Rings: The Fellowship of the Ring
The Return of the King
The Lord of the Rings
Kull the Conqueror
Shadowless Sword
Night Watch
Highlander: The Search for Vengeance
Hercules in the Haunted World
Hot News


In [ ]:
import pickle

with open('cosine_sim_matrix.pkl', 'wb') as f:
    pickle.dump(cosineSimilarities, f)

In [ ]:
df.to_csv('movies.csv')